In [1]:
!rm  -f ~/.letta/sqlite.db

In [2]:
import letta
from letta.schemas.memory import ChatMemory

import chromadb

from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

import os
import sys
super_directory = os.path.abspath('..')
sys.path.append(super_directory)

from modules.prompts import sanjay_persona, system_prompt

In [3]:
# Creating the letta_client and setting default LLM and Embedding Model
letta_client = letta.create_client()

letta_client.set_default_llm_config(letta.LLMConfig.default_config("gpt-4o-mini")) 
letta_client.set_default_embedding_config(letta.EmbeddingConfig.default_config("text-embedding-ada-002")) 

In [4]:
# Setting agent name
agent_name = 'dr_sanjay_sarma'

In [ ]:
# Getting the vector_db with Sanjay Sarma's information
chroma_client = chromadb.PersistentClient(path='../chromadb')
sanjay_collection = chroma_client.get_collection(name='sanjay_sarma')

vdb_embed_model = OpenAIEmbeddings(model='text-embedding-ada-002')

vectordb = Chroma(
    collection_name='sanjay_sarma',
    embedding_function=vdb_embed_model,
    persist_directory="../chromadb")

In [6]:
# Function as tool
def get_sanjay_information(query: str):
    """
    Retrieves relevant information related to a query.

    Args:
        query (str): The query string for which information is being retrieved.

    Returns:
        str: The concatenated content of the relevant search results based on the query.
    """
    from langchain_openai import OpenAIEmbeddings
    from langchain.vectorstores import Chroma
    
    vdb_embed_model = OpenAIEmbeddings(model='text-embedding-ada-002')
    vectordb = Chroma(
        collection_name='sanjay_sarma',
        embedding_function=vdb_embed_model,
        persist_directory="../chromadb")
    
    text = ""
    results = vectordb.similarity_search(query, k=5)
    for result in results:
        text += result.page_content
    return text

get_info_tool = letta_client.create_tool(get_sanjay_information)

In [ ]:
# Create sanjay agent
agent_state = letta_client.create_agent(
    name=agent_name,
    tool_ids=[get_info_tool.id],
    memory=ChatMemory(
        human="",
        persona=sanjay_persona
    )
)

print(f"Created agent with name {agent_state.name} and unique ID {agent_state.id}")

In [8]:
# Updating the system prompt
agent_state.system = system_prompt

In [ ]:
# Getting and printing response
user_message = "which story are you referring to that inspired your education"

response = letta_client.send_message(
    agent_id=agent_state.id, 
    message=user_message, 
    role="user" 
)
response

In [10]:
# import json

# for message in response.dict()['messages']:
#     if message.get('message_type') == 'tool_call_message' and message.get('tool_call', {}).get('name') == 'send_message':
#         message_reply = message['tool_call']['arguments']
        
#         print(json.loads(message_reply)['message'])